# Example for using SCD Transformer and Compute with ConDynS

We demonstrate here how to use SCD Transformer for writing SCDs with your custom prompts. Then, we show how to compute ConDynS.

In [1]:
import os
from convokit import Corpus, download
from convokit.convo_similarity import SCD
from convokit.convo_similarity.condyns import ConDynS
from convokit.genai import GenAIConfigManager

/reef/kz88/convokit/ConvoKit/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-19 04:53:21.200289: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-19 04:53:21.281305: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-19 04:53:22.808259: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may

In [2]:
corpus = Corpus(filename=download("friends-corpus"))

Dataset already exists at /reef/kz88/convokit/download_corpus/friends-corpus


## Write SCD and SoP with SCDWriter

In [ ]:
### Config your GenAI API keys
config = GenAIConfigManager()

# Set up Google Cloud configuration for Gemini (Vertex AI)
config.set_google_cloud_config("YOUR PROJECT", "YOUR LOCATION")

In [4]:
### Define your own formatter function for your data
def format_friends_transcript_from_convokit(convo):
    utt_lst = convo.get_utterance_ids()
    speaker_ids = {}
    transcript = ""
    for utt_id in utt_lst:
        utt = corpus.get_utterance(utt_id)
        if "TRANSCRIPT_NOTE" not in utt.speaker.id:
            if utt.speaker.id not in speaker_ids:
                speaker_ids[utt.speaker.id] = 1 + len(speaker_ids)
            transcript += "Speaker"+str(speaker_ids[utt.speaker.id]) + " : " + utt.text+ "\n\n"
    return transcript

In [5]:
### Prepare your own prompt for writing the SCD with your data
friends_summary_prompt = """
Write a short summary capturing the trajectory of a casual conversation. 
Do not include specific topics, events, or arguments from the conversation. The style you should avoid is illustrated in 
Example Sentence 1: “Speaker1 said they had a difficult day at work, and mentioned that their boss was unfair. Speaker2 listened and agreed that bosses can be tough, then suggested they go out for dinner to forget about it..” Instead, you should include indicators of sentiments (e.g., warmth, empathy, humor, nostalgia, vulnerability, support), individual intentions (e.g., building rapport, offering reassurance, seeking validation, self-disclosure, active listening, gentle disagreement, creating distance), and conversational strategies (if any) such as “collaborative storytelling,” “inside jokes,” “mirroring emotions,” and “affectionate teasing.” 
The following sentences demonstrate the style you should follow: 
Example Sentence 2: “Both speakers have similar feelings and appeared mutually supportive. Speaker1 initiates with a moment of self-disclosure, and Speaker2 responds with empathy and validation. Both speakers build on this exchange, strengthening their rapport.” 
Example Sentence 3: “The two speakers connected with back-and-forth affectionate teasing. Throughout the conversation, they kept building on each other's humor with playful remarks, creating a lighthearted and comfortable discussion.” Overall, the trajectory summary should capture the key moments where the emotional connection of the conversation notably changes. Here is an example of a complete trajectory summary: The conversation begins with two speakers exchanging neutral, surface-level comments. Speaker1 then shifts the tone by sharing a personal anecdote, prompting Speaker2 to respond with warmth and empathy. Speaker1 elaborates on their story and their need, but Speaker2 does not extend their support but retracts it. 
Now, provide the trajectory summary for the following conversation. 
Conversation Transcript: {transcript}. 
Now, summarize this conversation. Remember, do not include specific topics, claims, or arguments from the conversation. Instead, try to capture the speakers' sentiments, intentions, and conversational/persuasive strategies. Limit the trajectory summary to 80 words. 
Trajectory Summary:"""

Initialize your SCD transformer

In [6]:
MODEL_PROVIDER = "gemini"

In [7]:
scd_transformer = SCD(
        model_provider=MODEL_PROVIDER,
        config=config,
        model="gemini-2.0-flash-001",
        custom_scd_prompt=friends_summary_prompt,
        custom_prompt_dir="friends_prompts",
        generate_scd=True,
        generate_sop=True,
        scd_metadata_name="machine_scd",
        sop_metadata_name="machine_sop",
        conversation_formatter=format_friends_transcript_from_convokit
    )

In [8]:
conversation_ids = list(corpus.get_conversation_ids())[:2]
selector = lambda conv: conv.id in conversation_ids

In [9]:
corpus = scd_transformer.transform(corpus, selector=selector)

In [10]:
convo = corpus.get_conversation(conversation_ids[0])
print("SCD: ", convo.meta["machine_scd"])
print("SoP: ", convo.meta["machine_sop"])

SCD:  The conversation begins with playful teasing and lighthearted banter, establishing a comfortable, familiar dynamic. The mood shifts as one speaker expresses vulnerability, prompting initial empathy and offers of comfort from others. However, the speaker's distress escalates, leading to shared commiseration mixed with humor as a coping mechanism. A new speaker arrives, introducing a fresh wave of emotional turmoil and a need for support, which the group readily provides, solidifying their bond through shared experiences and understanding.

SoP:  {'0': 'The conversation begins with playful teasing and lighthearted banter.', '1': 'A comfortable, familiar dynamic is established.', '2': 'One speaker expresses vulnerability.', '3': 'Initial empathy and offers of comfort are provided by others.', '4': 'The speakers distress escalates.', '5': 'Shared commiseration mixed with humor emerges as a coping mechanism.', '6': 'A new speaker arrives.', '7': 'The new speaker introduces a fresh wav

## Compute ConDynS Score


In [11]:
condyns = ConDynS(model_provider=MODEL_PROVIDER, config=config)

In [12]:
convo_id1 = conversation_ids[0]
convo_id2 = conversation_ids[1]
convo1 = corpus.get_conversation(convo_id1)
convo2 = corpus.get_conversation(convo_id2)

transcript1 = format_friends_transcript_from_convokit(convo1)
transcript2 = format_friends_transcript_from_convokit(convo2)
sop1 = convo1.meta["machine_sop"]
sop2 = convo2.meta["machine_sop"]

result = condyns.compute_bidirectional_similarity(transcript1, transcript2, sop1, sop2)
score = condyns.compute_score_from_results(result)
print("ConDynS Result: ", result)
print("\n\nConDynS Score: ", score)

ConDynS Result:  [{'0': {'analysis': 'The conversation starts with playful guessing, fitting the description of lighthearted banter.', 'score': 0.7}, '1': {'analysis': 'The familiar dynamic is evident in the quick back-and-forth and shared jokes.', 'score': 0.8}, '2': {'analysis': 'Speaker4 expresses vulnerability about not wanting to be a shoe, a clear expression of vulnerability.', 'score': 1.0}, '3': {'analysis': 'Speaker1 offers comfort by telling Speaker4 to breathe, showing initial empathy.', 'score': 0.9}, '4': {'analysis': 'Speaker4s distress escalates as she talks about not needing the money but then retracts it.', 'score': 0.8}, '5': {'analysis': 'Speaker5 attempts to lighten the mood with a silly song, showing shared commiseration with humor.', 'score': 0.7}, '6': {'analysis': 'Speaker7, Paul, arrives, marking the arrival of a new speaker.', 'score': 1.0}, '7': {'analysis': 'Pauls arrival introduces a new dynamic, especially with Joeys inappropriate comment, creating emotion